## H3-Based MDP
Note that in the code we use $\gamma$ to represent the scaler for the station density reward function. However, the reason in the paper we represent it as $\delta$ is so that its not confusing to separate the $\delta$ in the context of the reward function or the $\delta$ used as a discount factor for Value Iteration.

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import networkx as nx
import h3
from shapely.geometry import Point
from mdptoolbox.mdp import ValueIteration

In [ ]:
# Load the hexagonal geojson file
hex_gdf = gpd.read_file("geojson\\hex_dataset.geojson")

# Ensure necessary columns exist
if "avg_imd_score" not in hex_gdf.columns or "total_population" not in hex_gdf.columns:
    raise ValueError("Missing column")

In [ ]:
# Declare states and num_states
hex_indices = hex_gdf["h3_index"]
states = list(hex_indices)
num_states = len(states)

In [ ]:
# Load existing stations
existing_stations_gdf = gpd.read_file("geojson\\existing_bike_stations.geojson")

if hex_gdf.crs != existing_stations_gdf.crs:
    existing_stations_gdf = existing_stations_gdf.to_crs(hex_gdf.crs)

existing_station_coords = existing_stations_gdf.geometry.apply(lambda geom: (geom.y, geom.x))
resolution = h3.get_resolution(states[0])
existing_station_hexes = set(h3.latlng_to_cell(lat, lon, resolution) for lat, lon in existing_station_coords)

# Build a reverse lookup for station hexes
station_density_lookup = {}

for h in states:
    neighbors = h3.grid_disk(h, 1)
    count = sum(1 for n in neighbors if n in existing_station_hexes)
    station_density_lookup[h] = count

In [ ]:
# Create an adjacency graph based on H3 neighbors
G = nx.Graph()
hex_indices = hex_gdf["h3_index"]
hex_indices_values = hex_indices.values  # for faster lookup

for h in hex_indices:
    neighbors = h3.grid_disk(h, 1)
    for n in neighbors:
        if n in hex_indices_values:
            G.add_edge(h, n)


In [ ]:
# Initialise rewards
rewards = np.zeros(num_states)

# Extract population and IMD arrays for normalization
all_pop = hex_gdf["total_population"]
all_imd = hex_gdf["avg_imd_score"]

min_pop, max_pop = all_pop.min(), all_pop.max()
min_imd, max_imd = all_imd.min(), all_imd.max()

# Tunable weights for demand vs equity
alpha = 0.5  # weight for population
beta = 0.5   # weight for deprivation (equity)
gamma = 0.4 # Station Density Weight

max_station_density = max(station_density_lookup.values()) or 1 # avoid division by zero

for i, h in enumerate(states):
    row = hex_gdf[hex_gdf["h3_index"] == h]
    if not row.empty:
        imd_score = row["avg_imd_score"].values[0]
        population = row["total_population"].values[0]

        norm_pop = (population - min_pop) / (max_pop - min_pop)
        norm_imd = (imd_score - min_imd) / (max_imd - min_imd)

        # Less dense areas get higher rewards
        station_density = station_density_lookup[h]
        norm_inverse_density = 1 - (station_density / gamma)

        reward = (
            alpha * norm_pop
            + beta * norm_imd
            + gamma * norm_inverse_density
        )

        rewards[i] = reward

In [ ]:
num_actions = 2
transition_matrix = np.zeros((num_actions, num_states, num_states))

# Action 0: Do nothing
for i in range(num_states):
    transition_matrix[0, i, i] = 1.0

# Action 1: Place a station
for i, h in enumerate(states):
    neighbors = h3.grid_disk(h, 1)
    valid_neighbors = [n for n in neighbors if n in states]
    prob = 1 / len(valid_neighbors) if valid_neighbors else 0
    for n in valid_neighbors:
        j = states.index(n)
        transition_matrix[1, i, j] = prob
    transition_matrix[1, i, i] += 1.0  # self-loop

# Normalize transitions
for a in range(num_actions):
    for s in range(num_states):
        row_sum = np.sum(transition_matrix[a, s, :])
        if row_sum > 0:
            transition_matrix[a, s, :] /= row_sum

In [ ]:
allowed_hexes = set()
for h in existing_station_hexes:
    allowed_hexes.update(h3.grid_disk(h, 3))  # within 3 rings (525m, which was chosen because people are only willing to walk around 400-500m from a bike station to their destination https://www.welovecycling.com/wide/2020/04/30/how-far-are-you-willing-to-walk-for-bike-sharing/)

filtered_states = [h for h in states if h in allowed_hexes]

if not filtered_states:
    raise ValueError("No candidate hexagons remain after filtering.")

filtered_indices = [states.index(h) for h in filtered_states]
filtered_rewards = rewards[filtered_indices]
filtered_transition_matrix = transition_matrix[:, filtered_indices, :][:, :, filtered_indices]

# Re-normalize transition matrix after slicing
for a in range(filtered_transition_matrix.shape[0]):
    for s in range(filtered_transition_matrix.shape[1]):
        row_sum = np.sum(filtered_transition_matrix[a, s, :])
        if row_sum > 0:
            filtered_transition_matrix[a, s, :] /= row_sum
        else:
            # self-loop if no transitions
            filtered_transition_matrix[a, s, s] = 1.0


In [ ]:
mdp = ValueIteration(filtered_transition_matrix, filtered_rewards, discount=0.9)
mdp.run()

In [ ]:
best_hex_indices = np.argsort(mdp.V)[-5:]
best_hexes = [filtered_states[i] for i in best_hex_indices]

best_locations = hex_gdf[hex_gdf["h3_index"].isin(best_hexes)][
    ["h3_index", "geometry", "total_population", "avg_imd_score"]
]

best_locations.to_file("Results\\best_hexagons.geojson", driver="GeoJSON")
best_locations.drop(columns="geometry").to_csv("Results\\best_hexagons.csv", index=False)

print("Results saved to 'best_hexagons.geojson' and 'best_hexagons.csv'")

## Voronoi MDP
One difference in methdology for the Voronoi-based MDP is that we need to do greedy selection post-processing to prevent the stations from clustering together. This is because the Voronoi cells significantly vary in size and shape, unlike H3 hexagons that are uniform and regularly spaced, which naturally have spatial separation.

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.neighbors import BallTree
import mdptoolbox
from geopy.distance import geodesic

In [2]:
# Import and normalise data
candidate_df = pd.read_csv("Dataset\\voronoi_candidates.csv")

candidate_df['norm_pop'] = (candidate_df['Population'] - candidate_df['Population'].min()) / (candidate_df['Population'].max() - candidate_df['Population'].min())
candidate_df['norm_imd'] = (candidate_df['IMD Score'] - candidate_df['IMD Score'].min()) / (candidate_df['IMD Score'].max() - candidate_df['IMD Score'].min())
candidate_df['norm_density'] = candidate_df['Density Weight'] / candidate_df['Density Weight'].max()

In [3]:
# Same weights as h3-based MDP
alpha = 0.5
beta = 0.5
gamma = 0.4

reward_base = (
    alpha * candidate_df['norm_pop'] +
    beta * candidate_df['norm_imd'] -
    gamma * candidate_df['norm_density']
).values

In [4]:
# Build graph from candidate location (r=500m)
coords_rad = np.deg2rad(candidate_df[['lat', 'lon']].values)
tree = BallTree(coords_rad, metric='haversine')
G = nx.Graph()
for i, coord in enumerate(coords_rad):
    indices = tree.query_radius([coord], r=500 / 6371000)[0]
    for j in indices:
        if i != j:
            G.add_edge(i, j)
neighbors = {i: list(G.neighbors(i)) for i in range(len(candidate_df))}

In [5]:
# Define MDP
n = len(candidate_df)
A = 2  # 0 = don't build, 1 = build
P = [np.eye(n) for _ in range(A)]  # identity transitions

R = np.zeros((n, A))
R[:, 0] = 0
R[:, 1] = reward_base.copy()
for i in range(n):
    if neighbors[i]:
        neighbor_penalty = np.mean([reward_base[j] for j in neighbors[i]])
        R[i, 1] -= 0.3 * neighbor_penalty * len(neighbors[i]) / max(len(nbrs) for nbrs in neighbors.values())


In [6]:
vi = mdptoolbox.mdp.ValueIteration(P, R, discount=0.9)
vi.run()

candidate_df['Value'] = vi.V
candidate_df['Policy'] = vi.policy

In [7]:
# Greedy post-processing to prevent spatial clustering

num_stations_to_build = 20
min_distance_m = 500
policy_df = candidate_df[candidate_df["Policy"] == 1].copy()
policy_df = policy_df.sort_values("Value", ascending=False)

selected_stations = []
for _, row in policy_df.iterrows():
    current_point = (row["lat"], row["lon"])
    too_close = False
    for sel in selected_stations:
        sel_point = (sel["lat"], sel["lon"])
        if geodesic(current_point, sel_point).meters < min_distance_m:
            too_close = True
            break
    if not too_close:
        selected_stations.append(row)
    if len(selected_stations) == num_stations_to_build:
        break

In [8]:
build_df = pd.DataFrame(selected_stations)
build_df["build_rank"] = np.arange(1, len(build_df) + 1)
build_df.to_csv("station_impact_ranking.csv", index=False)
print(build_df[["lat", "lon", "Value", "build_rank"]])

            lat       lon     Value  build_rank
1073  51.526790 -0.168740  7.728729           1
771   51.535840 -0.034040  7.222046           2
4658  51.520940 -0.177510  7.067878           3
5664  51.545976 -0.017493  6.807980           4
4909  51.515960 -0.036150  6.703326           5
4966  51.518290 -0.024390  6.492273           6
2534  51.520160 -0.062090  6.317756           7
1731  51.531670 -0.057770  6.243930           8
3973  51.535220 -0.074180  5.996592           9
3544  51.520450 -0.185960  5.982743          10
1686  51.533090 -0.064850  5.982435          11
2152  51.535190 -0.137040  5.981712          12
5798  51.532229 -0.084802  5.925009          13
3920  51.497970 -0.076330  5.923527          14
5543  51.512870 -0.057790  5.877344          15
5420  51.515910 -0.045700  5.837042          16
5795  51.533170 -0.096540  5.827771          17
1566  51.528030 -0.095260  5.804530          18
4951  51.523860 -0.024200  5.791032          19
3493  51.535240 -0.110940  5.775788     